## Problem Statment

You have been provided a dataset that consists of echo-location clicks of two types of whales, namely, Gervais and Cuviers. In this assignment, your task is to classify the different types whales using Gradient Boosting with the help of the XGBoost library. You are expected to fill in functions that would complete this task. We use XGBoost here instead of GradientBoostedTrees in Spark because XGBoost running on a single machine is much faster than Spark running on 10 machines.

The data files were preprocessed on PySpark (10 nodes) cluster. The code for the same can be found in Data_Processing_Whales.ipynb. The preprocessed data is a numpy array with `4175` rows (for the 10mb file) with following columns (zero-indexed):
* Col 0-9: projections on first 10 eigen vectors
* Col 10: rmse
* Col 11: peak2peak
* Col 12: label (`0 if row.species==u'Gervais' else 1`)

You can also refer to XGBoost_Whales.ipynb under for more details on the XGBoost Analysis before you attempt this assignment.

Both Data_Processing_Whales.ipynb and XGBoost_Whales.ipynb can be found under XGBoost directory that was uploaded in edX as a part of "Notebooks for weeks 7 & 8". 

## XGBoost - Theory

A brief overview of gradient boosting in XGBoost can be found here:

* http://xgboost.readthedocs.io/en/latest/model.html
* http://xgboost.readthedocs.io/en/latest/python/python_intro.html



Use the XGBoost API for training and predicting scores: 

* http://xgboost.readthedocs.io/en/latest/python/python_api.html

#### Main API

* `xgboost.train` is the learning API that trains the Gradient Boosting Model,
   * The main parameters are:
      * **plst** – XGBoost parameter list
      * **dtrain** – Data to be trained
      * **num_round** – Number of iterations of boosting. (default: 100)
      * **evallist** – List of items to be evaluated during training
      * **verbose_eval** - This can be used to control how much information the train function prints. You might want to set to **False** to avoid printing logs.
* `bst.predict` is the API that makes score predictions
   * The main parameters are:
      * **dtest** – Test Data
      * **dtrain** – Data to be trained
      * **ntree_limit** – Limit number of trees in the prediction (Use: ntree_limit=bst.best_ntree_limit)
      * **output_margin** - Whether to output the raw untransformed margin value (Use: output_margin=True)

## Notebook Setup

### Importing Required Libraries

In [1]:
%matplotlib inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import random

### Loading Data

In [2]:
with open('Data/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('Data/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('Data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('Data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### Setting Parameters for XG Boost
* Maximum Depth of the Tree = 3 _(maximum depth of each decision trees)_
* Step size shrinkage used in update to prevents overfitting = 0.3 _(how to weigh trees in subsequent iterations)_
* Evaluation Criterion= Maximize Loglikelihood according to the logistic regression _(logitboost)_
* Maximum Number of Iterations = 1000 _(total number trees for boosting)_
* Early Stop if score on Validation does not improve for 5 iterations

[Full description of options](https://xgboost.readthedocs.io/en/latest//parameter.html)

In [3]:
#You can change this cell if you wish to, but you aren't expected to
def xgboost_plst():
    param = {}
    param['max_depth']= 3   # depth of tree
    param['eta'] = 0.3      # shrinkage parameter
    param['silent'] = 1     # not silent
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'

    plst = param.items()
    return plst

## Exercises

### Computing the score ranges

The function <font color="blue">calc_stats</font> takes the xgboost margin scores as input and returns two numpy arrays min_scr, max_scr which are calculated as follows:

1. **min_scr**: mean - (3 $\times$ std)
2. **max_scr**: mean + (3 $\times$ std)

Here the input margin scores, represents the processed XGBoost margin scores obtained from the <font color="blue">bootstrap_pred</font> function. Each row represents the various scores for a specific example in an iteration and your <font color="blue">calc_stats</font> function is supposed to compute the **min_scr** and **max_scr** as defined for each example. So in the example below, we take a scenario where we have 3 examples which have 4 values each (From 4 bootstrap iterations).


**<font color="magenta" size=2>Example Input</font>**
``` python
[[-0.22 -0.19 -0.17 -0.13][-0.1 -0.05 0.02 0.10][0.03 0.11 0.12 0.15]]
```

Output: min_scr (numpy array), max_scr (numpy array)

**<font color="blue" size=2>Example Output</font>**
``` python
(array([-0.28 -0.23 -0.03]),
 array([-0.08  0.22  0.24]))
```

**Note**: Ensure you round the values in the numpy arrays to two decimal places

In [24]:
def calc_stats(margin_scores):
    ###
    ### YOUR CODE HERE
    ###
    mean = np.mean(margin_scores, axis = 1)
    std = np.std(margin_scores, axis = 1)
    return np.around(mean - 3 * std, 2), np.around(mean + 3 * std, 2)

In [25]:
margin_score = np.array([[-0.22, -0.19, -0.17, -0.13], [-0.1, -0.05, 0.02, 0.10], [0.03, 0.11, 0.12, 0.15]])
min_score, max_score = calc_stats(margin_score)
assert type(min_score) == np.ndarray, 'Incorrect Return type'
assert type(max_score) == np.ndarray, 'Incorrect Return type'

In [26]:
assert (min_score == np.array([-0.28, -0.23, -0.03])).all(), "Incorrect return value"

In [27]:
assert (max_score == np.array([-0.08,  0.22,  0.24])).all(), "Incorrect return value"

In [8]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [9]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Calculating predictions

Based on the ranges for each of the examples, i.e, (min_scr, max_scr), we can predict whether it's a Gervais or a Cuvier. Since all our scores will be between -1 and +1, we use 0 as the margin line. All examples which are on the left side of the margin, can be classified as Cuvier's and all which are on the right side can be classified as Gervais. However, since we take margin scores from a set of bootstraps for each example, we use the minimum and maximum score arrays to predict to determine the classification.

The function <font color="blue">predict</font> takes the minimum score array and maximum score array and returns predictions as follows:

1. If respective minimum score and maximum score values are less than 0, predict -1 (**Cuvier's**)
2. If respective minimum score value is less than 0 and maximum score value is greater than 0, predict 0 (**Unsure**)
3. If respective minimum score and maximum score values are greater than 0, predict 1 (**Gervais**)

**<font color="magenta" size=2>Example Input</font>**
``` python
min_scr (numpy array) = [-0.78 -0.68 -0.6 -0.53 -0.47 -0.42 -0.32 -0.21 -0.07 0.22]

max_scr (numpy array) = [-0.49 -0.39 -0.33 -0.25 -0.2 -0.11 -0.04 0.1 0.3 0.51]
```
Output: pred (numpy array of predictions)

**<font color="blue" size=2>Example Output</font>**
``` python
[-1 -1 -1 -1 -1 -1 -1  0  0  1]
```

In [28]:
def predict(min_scr, max_scr):
    ###
    ### YOUR CODE HERE
    ###
    min_predict = np.where(min_scr < 0, -1, 0)
    max_predict = np.where(max_scr < 0, 0, 1)
    return min_predict + max_predict

In [29]:
max_s = np.array([-0.49, -0.39, -0.33, -0.25, -0.2, -0.11, -0.04, 0.1, 0.3, 0.51])
min_s = np.array([-0.78, -0.68, -0.6, -0.53, -0.47, -0.42, -0.32, -0.21, -0.07, 0.22])
pred = predict(min_s, max_s)
true_pred = np.array([-1, -1, -1, -1, -1, -1, -1, 0, 0, 1])


In [30]:
assert type(pred) == np.ndarray, 'Incorrect return type'

In [31]:
assert (pred == true_pred).all(), 'Incorrect return value'

### Calculating scores

The function <font color="blue">bootstrap_pred</font> takes as input:

1. **Training set**
1. **Test set**
1. **n_bootstrap** Number of bootstrap samples that run XGBoost and trains one part of the sample set.
1. **minR, maxR** two numbers such that $0 < minR < maxR < 1$ that define the fractions of the `n_bootstrap` scores that define the range.
1. **bootstrap_size** - Number of bootstrap samples on which you will run XGBoost.
1. **num_round** - Number of iterations for running xgboost
1. **plst** - Parameter List

The output should be a confidence interval for each example in the test set. Together with a prediction that is `Gervais / Cuviers / Unsure`. The prediction `unsure` is to be output if the confidence interval contains the point 0.
After generating the confidence intervals, the function <font color="blue">predict</font> can be used to make predictions.

**Procedure**

Repeat the given procedure for n_bootstrap number of iterations:

For **n_bootstrap** iterations:
* Sample **boostrap_size** indices from the training set **with replacmennt**
* Create train and test data matrices (dtrain, dtest) using xgb.DMatrix(X_sample, label=y_sample)
* Initialise the evallist parameter [(dtrain, 'train'), (dtest, 'eval')]
* Train the model using the XGBoost train API and make score predictions using bst.predict object returned by XGB train API. Ensure you set **output_margin=True** to get raw untransformed output scores.
* Normalize them by dividing them with the normalizing factor as max(scores) - min(scores) and round these values to a precision of two decimal places

Then: 
* For each individual example, remove scores below the minRth percentile and greater than maxRth percentile (sort for each example if necessary)
* Call the calc_stats function to compute min_scr and max_scr with the filtered margin scores as parameter
* Return the min_scr and max_scr computed by the **calc_stat** function using the margin scores.
      
**Note**: You can experiment by changing **n_bootstraps**, but it takes about 200 iterations to get consistent values.

In [80]:
def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, minR, maxR, bootstrap_size, \
                   num_round=100, plst=xgboost_plst()):
    ###
    ### YOUR CODE HERE
    ###
    all_preds = []
    for i in range(n_bootstrap):
        indices = np.random.choice(X_train.shape[0], bootstrap_size)
        X_sample = X_train[indices, :]
        y_sample = y_train[indices]
        dtrain = xgb.DMatrix(X_sample, label=y_sample)
        dtest = xgb.DMatrix(X_test, label=y_test)
        evallist = [(dtrain, 'train'), (dtest, 'eval')]
        bst = xgb.train(plst, dtrain, num_round, evallist)
        preds = bst.predict(dtest, output_margin=True)
        preds = np.around(preds/(max(preds) - min(preds)), 2)
        all_preds.append(preds)
    all_preds = np.array(all_preds).T
    lower_bound = int(n_bootstrap*minR)
    higher_bound = int(n_bootstrap*maxR)
    all_preds = all_preds[:,lower_bound: higher_bound]
    min_scr, max_scr = calc_stats(all_preds)
    return min_scr,max_scr

In [81]:
def process(X_train, X_test, y_train, y_test, n_bootstrap=100):
    min_scr, max_scr = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=n_bootstrap, \
                                            minR=0.1, maxR=0.9, bootstrap_size=len(X_train))
    pred = predict(min_scr, max_scr)
    return min_scr, max_scr, pred

#### Tests

How we test the function:
1. We have computed the average mid-point of the range of values and verify that this midpoint is present in the range computed by your function
2. We check that the length of the interval(max_scr-min_scr) is not more than twice the average length of the interval

In [82]:
sample_indices = np.load('Data/vis_indices.npy')
X_test_samp = X_test[sample_indices]
y_test_samp = np.array(y_test[sample_indices], dtype=int)
midpt = np.load('Data/vis_midpt.npy')
avg_length = np.load('Data/vis_avg_length.npy')
min_scr, max_scr, pred = process(X_train, X_test_samp, y_train, y_test_samp)
length = max_scr - min_scr

[0]	train-logloss:0.556681	eval-logloss:0.627987
[1]	train-logloss:0.479583	eval-logloss:0.584117
[2]	train-logloss:0.42965	eval-logloss:0.593812
[3]	train-logloss:0.394363	eval-logloss:0.556308
[4]	train-logloss:0.371759	eval-logloss:0.573538
[5]	train-logloss:0.352995	eval-logloss:0.545587
[6]	train-logloss:0.338188	eval-logloss:0.564343
[7]	train-logloss:0.323418	eval-logloss:0.56144
[8]	train-logloss:0.31383	eval-logloss:0.57312
[9]	train-logloss:0.304848	eval-logloss:0.595178
[10]	train-logloss:0.296081	eval-logloss:0.615124
[11]	train-logloss:0.289182	eval-logloss:0.632748
[12]	train-logloss:0.284802	eval-logloss:0.625966
[13]	train-logloss:0.277771	eval-logloss:0.673662
[14]	train-logloss:0.272437	eval-logloss:0.696293
[15]	train-logloss:0.267951	eval-logloss:0.710683
[16]	train-logloss:0.262874	eval-logloss:0.68156
[17]	train-logloss:0.258881	eval-logloss:0.704092
[18]	train-logloss:0.254952	eval-logloss:0.693478
[19]	train-logloss:0.249379	eval-logloss:0.682459
[20]	train-logl

[66]	train-logloss:0.136197	eval-logloss:0.756721
[67]	train-logloss:0.135256	eval-logloss:0.754392
[68]	train-logloss:0.133367	eval-logloss:0.787488
[69]	train-logloss:0.132752	eval-logloss:0.785227
[70]	train-logloss:0.131014	eval-logloss:0.795307
[71]	train-logloss:0.128737	eval-logloss:0.804176
[72]	train-logloss:0.126751	eval-logloss:0.795432
[73]	train-logloss:0.124595	eval-logloss:0.809988
[74]	train-logloss:0.12364	eval-logloss:0.785517
[75]	train-logloss:0.122467	eval-logloss:0.781956
[76]	train-logloss:0.121511	eval-logloss:0.782046
[77]	train-logloss:0.12049	eval-logloss:0.802748
[78]	train-logloss:0.120053	eval-logloss:0.812574
[79]	train-logloss:0.118858	eval-logloss:0.820747
[80]	train-logloss:0.117778	eval-logloss:0.827642
[81]	train-logloss:0.116482	eval-logloss:0.838107
[82]	train-logloss:0.115743	eval-logloss:0.843814
[83]	train-logloss:0.115082	eval-logloss:0.836513
[84]	train-logloss:0.113273	eval-logloss:0.857144
[85]	train-logloss:0.111858	eval-logloss:0.866046
[8

[31]	train-logloss:0.210011	eval-logloss:0.827786
[32]	train-logloss:0.205777	eval-logloss:0.848055
[33]	train-logloss:0.201821	eval-logloss:0.852753
[34]	train-logloss:0.198349	eval-logloss:0.8626
[35]	train-logloss:0.195023	eval-logloss:0.886016
[36]	train-logloss:0.193734	eval-logloss:0.883294
[37]	train-logloss:0.190788	eval-logloss:0.902264
[38]	train-logloss:0.188031	eval-logloss:0.907555
[39]	train-logloss:0.186707	eval-logloss:0.900628
[40]	train-logloss:0.185507	eval-logloss:0.907679
[41]	train-logloss:0.183574	eval-logloss:0.973182
[42]	train-logloss:0.182763	eval-logloss:0.963111
[43]	train-logloss:0.180983	eval-logloss:0.942479
[44]	train-logloss:0.180416	eval-logloss:0.942336
[45]	train-logloss:0.177824	eval-logloss:0.936142
[46]	train-logloss:0.175521	eval-logloss:0.932364
[47]	train-logloss:0.172115	eval-logloss:0.92528
[48]	train-logloss:0.169853	eval-logloss:0.89678
[49]	train-logloss:0.167502	eval-logloss:0.889741
[50]	train-logloss:0.165549	eval-logloss:0.889323
[51]

[96]	train-logloss:0.103279	eval-logloss:0.618531
[97]	train-logloss:0.101825	eval-logloss:0.615888
[98]	train-logloss:0.101151	eval-logloss:0.620464
[99]	train-logloss:0.099851	eval-logloss:0.61073
[0]	train-logloss:0.550316	eval-logloss:0.625238
[1]	train-logloss:0.470314	eval-logloss:0.572291
[2]	train-logloss:0.421773	eval-logloss:0.558086
[3]	train-logloss:0.3859	eval-logloss:0.559561
[4]	train-logloss:0.362583	eval-logloss:0.586257
[5]	train-logloss:0.344996	eval-logloss:0.58529
[6]	train-logloss:0.331387	eval-logloss:0.601613
[7]	train-logloss:0.318494	eval-logloss:0.625573
[8]	train-logloss:0.308889	eval-logloss:0.6256
[9]	train-logloss:0.299755	eval-logloss:0.67082
[10]	train-logloss:0.293694	eval-logloss:0.695088
[11]	train-logloss:0.285083	eval-logloss:0.704519
[12]	train-logloss:0.2774	eval-logloss:0.693037
[13]	train-logloss:0.270459	eval-logloss:0.733286
[14]	train-logloss:0.264823	eval-logloss:0.791127
[15]	train-logloss:0.260489	eval-logloss:0.824616
[16]	train-logloss:

[63]	train-logloss:0.134173	eval-logloss:0.726341
[64]	train-logloss:0.133524	eval-logloss:0.716669
[65]	train-logloss:0.131364	eval-logloss:0.711287
[66]	train-logloss:0.130885	eval-logloss:0.724557
[67]	train-logloss:0.130031	eval-logloss:0.756656
[68]	train-logloss:0.128309	eval-logloss:0.768338
[69]	train-logloss:0.126403	eval-logloss:0.813691
[70]	train-logloss:0.124221	eval-logloss:0.8108
[71]	train-logloss:0.123208	eval-logloss:0.810438
[72]	train-logloss:0.121495	eval-logloss:0.7747
[73]	train-logloss:0.119847	eval-logloss:0.779418
[74]	train-logloss:0.118532	eval-logloss:0.782322
[75]	train-logloss:0.117933	eval-logloss:0.780069
[76]	train-logloss:0.116534	eval-logloss:0.768813
[77]	train-logloss:0.115168	eval-logloss:0.755243
[78]	train-logloss:0.113629	eval-logloss:0.760301
[79]	train-logloss:0.113387	eval-logloss:0.754353
[80]	train-logloss:0.112531	eval-logloss:0.770445
[81]	train-logloss:0.111165	eval-logloss:0.772612
[82]	train-logloss:0.109708	eval-logloss:0.780412
[83]

...

[60]	train-logloss:0.1461	eval-logloss:0.834073
[61]	train-logloss:0.143697	eval-logloss:0.863351
[62]	train-logloss:0.141001	eval-logloss:0.861376
[63]	train-logloss:0.13979	eval-logloss:0.857741
[64]	train-logloss:0.137737	eval-logloss:0.841775
[65]	train-logloss:0.135667	eval-logloss:0.844168
[66]	train-logloss:0.133992	eval-logloss:0.804972
[67]	train-logloss:0.133258	eval-logloss:0.796515
[68]	train-logloss:0.13129	eval-logloss:0.79479
[69]	train-logloss:0.129253	eval-logloss:0.783243
[70]	train-logloss:0.127952	eval-logloss:0.791519
[71]	train-logloss:0.127257	eval-logloss:0.771282
[72]	train-logloss:0.125805	eval-logloss:0.766971
[73]	train-logloss:0.124714	eval-logloss:0.766945
[74]	train-logloss:0.123818	eval-logloss:0.775054
[75]	train-logloss:0.122236	eval-logloss:0.787579
[76]	train-logloss:0.121722	eval-logloss:0.807621
[77]	train-logloss:0.120312	eval-logloss:0.803279
[78]	train-logloss:0.119328	eval-logloss:0.779845
[79]	train-logloss:0.118628	eval-logloss:0.779865
[80]	

[25]	train-logloss:0.229294	eval-logloss:0.664813
[26]	train-logloss:0.225942	eval-logloss:0.665215
[27]	train-logloss:0.220937	eval-logloss:0.704123
[28]	train-logloss:0.217727	eval-logloss:0.689089
[29]	train-logloss:0.214258	eval-logloss:0.701116
[30]	train-logloss:0.213238	eval-logloss:0.698735
[31]	train-logloss:0.211165	eval-logloss:0.698595
[32]	train-logloss:0.209229	eval-logloss:0.694214
[33]	train-logloss:0.206117	eval-logloss:0.734465
[34]	train-logloss:0.204878	eval-logloss:0.716398
[35]	train-logloss:0.202673	eval-logloss:0.724319
[36]	train-logloss:0.201312	eval-logloss:0.733405
[37]	train-logloss:0.198074	eval-logloss:0.753785
[38]	train-logloss:0.196588	eval-logloss:0.737593
[39]	train-logloss:0.193325	eval-logloss:0.732548
[40]	train-logloss:0.189929	eval-logloss:0.739509
[41]	train-logloss:0.187973	eval-logloss:0.749771
[42]	train-logloss:0.18708	eval-logloss:0.755095
[43]	train-logloss:0.184697	eval-logloss:0.754525
[44]	train-logloss:0.183226	eval-logloss:0.72936
[4

[90]	train-logloss:0.100436	eval-logloss:0.822034
[91]	train-logloss:0.099112	eval-logloss:0.820215
[92]	train-logloss:0.098708	eval-logloss:0.811672
[93]	train-logloss:0.097622	eval-logloss:0.814994
[94]	train-logloss:0.096899	eval-logloss:0.807522
[95]	train-logloss:0.095916	eval-logloss:0.818169
[96]	train-logloss:0.094897	eval-logloss:0.817786
[97]	train-logloss:0.093836	eval-logloss:0.809366
[98]	train-logloss:0.092747	eval-logloss:0.817384
[99]	train-logloss:0.09248	eval-logloss:0.817269
[0]	train-logloss:0.555635	eval-logloss:0.598553
[1]	train-logloss:0.47568	eval-logloss:0.578215
[2]	train-logloss:0.427115	eval-logloss:0.561609
[3]	train-logloss:0.392972	eval-logloss:0.498698
[4]	train-logloss:0.370833	eval-logloss:0.509363
[5]	train-logloss:0.353992	eval-logloss:0.550387
[6]	train-logloss:0.339513	eval-logloss:0.579217
[7]	train-logloss:0.32873	eval-logloss:0.611178
[8]	train-logloss:0.317721	eval-logloss:0.668967
[9]	train-logloss:0.309499	eval-logloss:0.635306
[10]	train-lo

[55]	train-logloss:0.157593	eval-logloss:0.668428
[56]	train-logloss:0.156503	eval-logloss:0.668094
[57]	train-logloss:0.155424	eval-logloss:0.683751
[58]	train-logloss:0.153774	eval-logloss:0.686509
[59]	train-logloss:0.15315	eval-logloss:0.702433
[60]	train-logloss:0.151068	eval-logloss:0.694377
[61]	train-logloss:0.149964	eval-logloss:0.724481
[62]	train-logloss:0.148845	eval-logloss:0.751023
[63]	train-logloss:0.14648	eval-logloss:0.729242
[64]	train-logloss:0.144664	eval-logloss:0.725214
[65]	train-logloss:0.143199	eval-logloss:0.7213
[66]	train-logloss:0.141302	eval-logloss:0.708829
[67]	train-logloss:0.139691	eval-logloss:0.715244
[68]	train-logloss:0.137623	eval-logloss:0.711574
[69]	train-logloss:0.136918	eval-logloss:0.724979
[70]	train-logloss:0.135349	eval-logloss:0.749324
[71]	train-logloss:0.134287	eval-logloss:0.731776
[72]	train-logloss:0.133246	eval-logloss:0.731848
[73]	train-logloss:0.132948	eval-logloss:0.731896
[74]	train-logloss:0.13116	eval-logloss:0.740581
[75]	

[20]	train-logloss:0.243403	eval-logloss:0.534679
[21]	train-logloss:0.240195	eval-logloss:0.533857
[22]	train-logloss:0.237234	eval-logloss:0.558737
[23]	train-logloss:0.2345	eval-logloss:0.552301
[24]	train-logloss:0.231678	eval-logloss:0.56147
[25]	train-logloss:0.229639	eval-logloss:0.55175
[26]	train-logloss:0.224947	eval-logloss:0.539099
[27]	train-logloss:0.221251	eval-logloss:0.542559
[28]	train-logloss:0.219592	eval-logloss:0.56978
[29]	train-logloss:0.217939	eval-logloss:0.566168
[30]	train-logloss:0.214837	eval-logloss:0.515152
[31]	train-logloss:0.212566	eval-logloss:0.529072
[32]	train-logloss:0.209671	eval-logloss:0.528657
[33]	train-logloss:0.205573	eval-logloss:0.535131
[34]	train-logloss:0.203247	eval-logloss:0.523606
[35]	train-logloss:0.199085	eval-logloss:0.549934
[36]	train-logloss:0.195796	eval-logloss:0.550302
[37]	train-logloss:0.193752	eval-logloss:0.562878
[38]	train-logloss:0.192925	eval-logloss:0.562727
[39]	train-logloss:0.191736	eval-logloss:0.603752
[40]	

In [83]:
assert sum(min_scr <= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [84]:
assert sum(max_scr >= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [85]:
assert sum(length < 2*avg_length) >= (0.7 * len(sample_indices)), "Incorrect length of range (mean - 3*std) to (mean + 3*std)"

In [20]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [21]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [22]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [23]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [24]:
# Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [25]:
# Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [26]:
# Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [27]:
# Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###
